# Introduction to Word Embeddings

Sources:
https://www.gavagai.se/blog/2015/09/30/a-brief-history-of-word-embeddings/
http://sebastianruder.com/word-embeddings-1/index.html


## A Brief of History
Extracted from https://www.gavagai.se/blog/2015/09/30/a-brief-history-of-word-embeddings/

Word embeddings are a epresentational basis for downstream NLP tasks like text classification, document clustering, part of speech tagging, named entity recognition, sentiment analysis, and so on.  

Word embedding seems to be the dominating term at the moment, I often prefer the term distributional semantic model (since the underlying semantic theory is called [distributional semantics](https://en.wikipedia.org/wiki/Distributional_semantics)).  

There are also many other alternative terms in use, from the very general distributed representation to the more specific semantic vector space or simply word space.  

Methods for using automatically generated contextual features were developed more or less simultaneously around 1990 in several different research areas. One of the most influential early models was Latent Semantic Analysis/Indexing (LSA/LSI), developed in the context of information retrieval, and the precursor of today’s topic models.  

The most well-known of these are probably Self Organizing Maps (SOM) and Simple Recurrent Networks (SRN), of which the latter is the precursor to today’s neural language models.

Later developments are basically only refinements of these early models. Topic models are refinements of LSA, and include methods like probabilistic LSA (PLSA) and Latent Dirichlet Allocation (LDA).  

The main difference between these various models is the type of contextual information they use. LSA and topic models use documents as contexts, which is a legacy from their roots in information retrieval. Neural language models and distributional semantic models instead use words as contexts, which is arguably more natural from a linguistic and cognitive perspective. These different contextual representations capture different types of semantic similarity; the document-based models capture semantic relatedness (e.g. “boat” – “water”) while the word-based models capture semantic similarity (e.g. “boat” – “ship”).  

There is no qualitative difference between (current) predictive neural network models and count-based distributional semantics models. Rather, they are different computational means to arrive at the same type of semantic model; several recent papers have demonstrated both theoretically and empirically the correspondence between these different types of models. ([Österlund et al. (2015)](http://aclweb.org/anthology/D/D15/D15-1024.pdf), [Levy and Goldberg (2014)](https://levyomer.files.wordpress.com/2014/09/neural-word-embeddings-as-implicit-matrix-factorization.pdf))

A good bet is to use a factorized model – either using explicit factorization of a distributional semantic model (available in e.g. the PyDSM python library or the GloVe implementation), or using a neural network model like those implemented in word2vec – since they produce state of the art results and are robust across a wide range of semantic tasks ([Schnabel et al., 2015](http://aclweb.org/anthology/D/D15/D15-1036.pdf)).

Word embeddings are one of the few currently successful applications of unsupervised learning. Their main benefit arguably is that they don't require expensive annotation, but can be derived from large unannotated corpora that are readily available. Pre-trained embeddings can then be used in downstream tasks that use small amounts of labeled data.

## Word embedding models

Every feed-forward neural network that takes words from a vocabulary as input and embeds them as vectors into a lower dimensional space, which it then fine-tunes through back-propagation, necessarily yields word embeddings as the weights of the first layer, which is usually referred to as Embedding Layer.

We assume the following notational standards: We assume a training corpus containing a sequence of $T$ training words $w_1,w_2,w_3,⋯,w_T$ that belong to a vocabulary $V$ whose size is $|V|$. Our models generally consider a context of $n$ words. We associate every word with an input embedding $v_w$ (the eponymous word embedding in the Embedding Layer) with $d$ dimensions and an output embedding $v′_w$ (another word representation whose role will soon become clearer). We finally optimize an objective function $J_θ$ with regard to our model parameters $θ$ and our model outputs some score $f_θ(x)$ for every input $x$.  

### A note in language modelling

Language models generally try to compute the probability of a word $w_t$ given its $n−1$ previous words, i.e. $p(wt|w_{t−1},⋯w_{t−n+1})$. By applying the chain rule together with the Markov assumption, we can approximate the probability of a whole sentence or document by the product of the probabilities of each word given its n previous words:

$$p(w_1,⋯,w_T)=\prod_i p(w_i|w_{i−1},⋯,w_{i−n+1})$$

In n-gram based language models, we can calculate a word's probability based on the frequencies of its constituent n-grams:  

$$p(w_t|w_{t−1},⋯,w_{t−n+1})=\frac{count(w_{t−n+1},⋯,w_{t−1},w_t)}{count(w_{t−n+1},⋯,w_{t−1})}$$

More info about n-grams in these [slides](https://web.stanford.edu/class/cs124/lec/languagemodeling.pdf) from Stanford.

In neural networks, we achieve the same objective using the well-known softmax layer:  

$$p(w_t|w_{t−1},⋯,w_{t−n+1})=\frac{\exp(h^⊤v′_{w_t})}{\sum_{w_i∈V}exp(h^⊤v′_{w_i})}$$

The inner product $h^⊤v′_{w_t}$ computes the (unnormalized) log-probability of word $w_t$, which we normalize by the sum of the log-probabilities of all words in $V$. $h$ is the output vector of the penultimate network layer (the hidden layer in the feed-forward network in Figure 1), while $v′_w$ is the output embedding of word $w$, i.e. its representation in the weight matrix of the softmax layer. Note that even though $v′_w$ represents the word $w$, it is learned separately from the input word embedding $v_w$, as the multiplications both vectors are involved in differ ($v_w$ is multiplied with an index vector, $v′_w$ with $h$).

<figure text-align=center>
  <img src="nn_language_model-1.jpg">
</figure>  

Figure 1: A neural language model (Bengio et al., 2006)

Note that we need to calculate the probability of every word $w$ at the output layer of the neural network. To do this efficiently, we perform a matrix multiplication between $h$ and a weight matrix whose rows consist of $v′_w$ of all words $w$ in $V$. We then feed the resulting vector, which is often referred to as a logit, i.e. the output of a previous layer that is not a probability, with $d=|V|$ into the softmax, while the softmax layer "squashes" the vector to a probability distribution over the words in $V$.

In the state of the art algorithms the fully connected layer is replaced by an LSTM layer.

Using this softmax layer, the model tries to maximize the probability of predicting the correct word at every timestep t. The whole model thus tries to maximize the averaged log probability of the whole corpus:

$$J_θ=\frac{1}{T}\sum_{t=1}^T log p(w_t|w_{t−1},⋯,w_{t−n+1})$$

To sample words from the language model at test time, we can either greedily choose the word with the highest probability $p(w_t|w_{t−1}⋯w_{t−n+1})$ at every time step t or use beam search. We can do this for instance to generate arbitrary text sequences or as part of a sequence prediction task, where an LSTM is used as the decoder.

## Word2Vec

### Continuous bag-of-words (CBOW)

While a language model is only able to look at the past words for its predictions, as it is evaluated on its ability to predict each next word in the corpus, a model that just aims to generate accurate word embeddings does not suffer from this restriction. Mikolov et al. thus use both the n words before and after the target word wt to predict it as depicted in Figure 4. They call this continuous bag-of-words (CBOW), as it uses continuous representations whose order is of no importance.

<figure text-align=center>
    <img src="cbow.png">
</figure>
Figure 4: Continuous bag-of-words (Mikolov et al., 2013)

The objective function of CBOW in turn is only slightly different than the language model one:

$$J_θ=\frac{1}{T}\sum_{t=1}^T \log p(w_t|w_{t−n},⋯,w_{t−1},w_{t+1},⋯,w_{t+n})$$

Instead of feeding $n$ previous words into the model, the model receives a window of $n$ words around the target word $w_t$ at each time step $t$.

### Skip-gram

While CBOW can be seen as a precognitive language model, skip-gram turns the language model objective on its head: Instead of using the surrounding words to predict the centre word as with CBOW, skip-gram uses the centre word to predict the surrounding words as can be seen in Figure 5.

<figure text-align=center>
    <img src="skip-gram.png">
</figure>  
Figure 5: Skip-gram (Mikolov et al., 2013)

$$J_θ=\frac{1}{T}\sum_{t=1}^{T} \sum_{−n≤j≤n,j≠0} \log p(w_{t+j}|{w_t})$$

This is done using the following softmax as a probability model:

$$p(w_{t+j}|w_t)=\frac{exp(v^⊤_{w_t} v′_{w_{t+j}})}{\sum_{w_i∈V}exp(v^⊤_{w_t}v′_{w_i})}$$

Note that $v_{w_i}$ is the embedding of word i, thus a column of the embedding matrix. $V'$ matrix is the matrix to decode the embedding representation into the logits vector. From the logits vector we can calculate the co-occurrence probability.

The notation in Mikolov's paper differs slightly from ours, as they denote the centre word with $w_I$ and the surrounding words with $w_O$. If we replace $w_t$ with $w_I, w_{t+j}$ with $w_O$, and swap the vectors in the inner product due to its commutativity, we arrive at the softmax notation in their paper:

$$p(w_O|w_I)=\frac{\exp(v′^⊤_{w_O}v_{w_I})}{\sum^V_{w=1}\exp(v′^⊤_{w_v}v_{w_I})}$$

To train this softmax function is extremely computational expensive. For each training iteration the learning gradient must propagate not only through the desired target word output, but the al other outputs. This is computational expensive and the next Word2Vect papers are related to training optimizations.

## The softmax

Now new are going to take a deeper look at the softmax. For more insight please refer to [this link](http://cs231n.github.io/linear-classify/#softmax-classifier).  
In the Softmax classifier, the function mapping $f(x_i;W)=W x_i$ represents the unnormalized log probabilities for each class. The asociated loss is a cross-entropy loss that has the form:

$$L_i=-\log \left( \frac{\exp(f_{y_i})}{\sum_j \exp(f_j)} \right) $$ or equivalently $$L_i=-f_{y_i}+\log \sum_j \exp(f_j) $$

where we are using the notation $f_j$ to mean the j-th element of the vector of class scores $f$. As before, the full loss for the dataset is the mean of $L_i$ over all training examples together with a regularization term $R(W)$. The function $f_j(z)=\frac {\exp(z_j)}{\sum_k{\exp(z_k)}}$ is called the softmax function: It takes a vector of arbitrary real-valued scores (in $z$) and squashes it to a vector of values between zero and one that sum to one. The full cross-entropy loss that involves the softmax function might look scary if you’re seeing it for the first time but it is relatively easy to motivate.

### Information theory view. 

The cross-entropy between a “true” distribution $p$ and an estimated distribution $q$ is defined as:

$$H(p,q)=−\sum_x p(x) \log q(x)$$

The Softmax classifier is hence minimizing the cross-entropy between the estimated class probabilities ($q=\frac{\exp(f_{y_i}}{\sum_j \exp(f_j)}$ as seen above) and the “true” distribution, which in this interpretation is the distribution where all probability mass is on the correct class (i.e. $p=[0,…1,…,0]$ contains a single $1$ at the $y_i$ -th position.). Moreover, since the cross-entropy can be written in terms of entropy and the Kullback-Leibler divergence as $H(p,q)=H(p)+DKL(p||q)$, and the entropy of the delta function $p$ is zero, this is also equivalent to minimizing the KL divergence between the two distributions (a measure of distance). In other words, the cross-entropy objective wants the predicted distribution to have all of its mass on the correct answer.

### Probabilistic interpretation. 

Looking at the expression, we see that

$$ P(y_i∣x_i;W)=\frac{\exp(f_{y_i})}{\sum_j \exp (f_j)} $$  

can be interpreted as the (normalized) probability assigned to the correct label $y_i$ given the image $x_i$ and parameterized by $W$. To see this, remember that the Softmax classifier interprets the scores inside the output vector $f$ as the unnormalized log probabilities. Exponentiating these quantities therefore gives the (unnormalized) probabilities, and the division performs the normalization so that the probabilities sum to one. In the probabilistic interpretation, we are therefore minimizing the negative log likelihood of the correct class, which can be interpreted as performing Maximum Likelihood Estimation (MLE). A nice feature of this view is that we can now also interpret the regularization term $R(W)$ in the full loss function as coming from a Gaussian prior over the weight matrix $W$, where instead of MLE we are performing the Maximum a posteriori (MAP) estimation. We mention these interpretations to help your intuitions, but the full details of this derivation are beyond the scope of this class.

### Practical issues: Numeric stability

When you’re writing code for computing the Softmax function in practice, the intermediate terms $\exp(f_{y_i})$ and $\sum_j(f_j)$ may be very large due to the exponentials. Dividing large numbers can be numerically unstable, so it is important to use a normalization trick. Notice that if we multiply the top and bottom of the fraction by a constant $C$ and push it into the sum, we get the following (mathematically equivalent) expression:

$$\frac{\exp(f_{y_i})}{\sum_j{\exp(f_j)}}=\frac{C\exp(f_{y_i})}{C\sum_j{\exp(f_j)}}=\frac{\exp(f_{y_i}+\log C)}{\sum_j{\exp(f_j + \log C)}}$$  
We are free to choose the value of $C$. This will not change any of the results, but we can use this value to improve the numerical stability of the computation. A common choice for $C$ is to set $\log C=−\max_j(f_j)$. This simply states that we should shift the values inside the vector $f$ so that the highest value is zero. In code:

In [2]:
import numpy as np
f = np.array([123, 456, 789]) # example with 3 classes and each having large scores
p = np.exp(f) / np.sum(np.exp(f)) # Bad: Numeric problem, potential blowup
print(f)
print(p)
# instead: first shift the values of f so that the highest number is 0:
f -= np.max(f) # f becomes [-666, -333, 0]
p = np.exp(f) / np.sum(np.exp(f)) # safe to do, gives the correct answer
print(f)
print(p)

[123 456 789]
[ 0.  0. nan]
[-666 -333    0]
[5.75274406e-290 2.39848787e-145 1.00000000e+000]


/home/cselmo/.conda/envs/doctorado-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


## Aproximating the softmax

In this section we are going to take a look at the softmax function. If you want more detail about the softmax, please refer to [this link
http://cs231n.github.io/linear-classify/#softmax-classifier

We observed that mitigating the complexity of computing the final softmax layer has been one of the main challenges in devising better word embedding models, a commonality with machine translation (MT) ([Jean et al.](http://www.aclweb.org/anthology/P15-1001)) and language modelling ([Jozefowicz et al.](https://arxiv.org/pdf/1602.02410.pdf)).

There are several strategies that have been proposed to approximate the softmax. These approaches can be grouped into softmax-based and sampling-based approaches. Softmax-based approaches are methods that keep the softmax layer intact, but modify its architecture to improve its efficiency. Sampling-based approaches on the other hand completely do away with the softmax layer and instead optimise some other loss function that approximates the softmax.

We will explore Noise Contrastive Estimation, a Sampling-based approach to approximate the softmax. Then we will take a look to Negative Sampling. 

### Noise Contrastive Estimation

Noise Contrastive Estimation (NCE) ([Gutmann and Hyvärinen](http://www.cs.helsinki.fi/u/ahyvarin/papers/Gutmann10AISTATS.pdf)) is proposed by [Mnih and Teh](https://www.cs.toronto.edu/~amnih/papers/ncelm.pdf).
In this case, we train a model to differentiate the target word from noise. We can thus reduce the problem of predicting the correct word to a binary classification task, where the model tries to distinguish positive, genuine data from noise samples, as can be seen in Figure 4 below.

<img src="negative_sampling.png">

Figure 4: Noise Contrastive Estimation ([Stephan Gouws' PhD dissertation](https://pdfs.semanticscholar.org/presentation/f775/cea7cd7f368094a853ea396121950d37915c.pdf))

For every word $w_i$ given its context ci of n previous words $w_{t−1},⋯,w_{t−n+1}$ in the training set, we thus generate $k$ noise samples $\tilde{w}_{ik}$ from a noise distribution $Q$. We can sample from the unigram distribution of the training set. As we need labels to perform our binary classification task, we designate all correct words $w_i$ given their context $c_i$ as true ($y=1$) and all noise samples $\tilde{w}_{ik}$ as false ($y=0$).  

We can now use logistic regression to minimize the negative log-likelihood, i.e. cross-entropy of our training examples against the noise (conversely, we could also maximize the positive log-likelihood as some papers do):

$$J_θ=−\sum_{w_i∈V}\left[ \log P(y=1|w_i,c_i)+ k \mathbb{E}_{\tilde{w}_{ik}\sim Q} \left[ \log P(y=0|\tilde{w}_{ij},c_i) \right] \right]$$

Instead of computing the expectation $\mathbb{E}_{\tilde{w}_{ik}}∼Q$ of our noise samples, which would still require summing over all words in $V$ to predict the normalised probability of a negative label, we can again take the mean with the Monte Carlo approximation:

$$J_θ=−\sum_{w_i∈V}\left[\log P(y=1|w_i,c_i) + k \sum^k_{j=1}\frac{1}{k} \log P(y=0|\tilde{w}_{ij},c_i)\right]$$

Which reduces to:

$$J_θ=−\sum_{w_i∈V}\left[\log P(y=1|w_i,c_i) +  \sum^k_{j=1} \log P(y=0|\tilde{w}_{ij},c_i)\right]$$

By generating $k$ noise samples for every genuine word wi given its context $c$, we are effectively sampling words from two different distributions: Correct words are sampled from the empirical distribution of the training set $P_{train}$ and depend on their context $c$, whereas noise samples come from the noise distribution $Q$. We can thus represent the probability of sampling either a positive or a noise sample as a mixture of those two distributions, which are weighted based on the number of samples that come from each:

$$P(y,w|c)=\frac{1}{k+1}P_{train}(w|c)+\frac{k}{k+1}Q(w)$$

Given this mixture, we can now calculate the probability that a sample came from the training Ptrain distribution as a conditional probability of y given $w$ and $c$:

$$P(y=1|w,c)=\frac{\frac{1}{k+1}P_{train}(w|c)}{\frac{1}{k+1}P_{train}(w|c)+\frac{k}{k+1}Q(w)}$$

which can be simplified to:

$$P(y=1|w,c)=\frac{P_{train}(w|c)}{P_{train}(w|c)+k Q(w)}$$

As we don't know $P_train$ (which is what we would like to calculate), we replace $P_train$ with the probability of our model $P$:

$$P(y=1|w,c)=\frac{P(w|c)}{P(w|c)+k Q(w)}$$

The probability of predicting a noise sample $(y=0)$ is then simply $P(y=0|w,c)=1−P(y=1|w,c)$. Note that computing $P(w|c)$, i.e. the probability of a word w given its context $c$ is essentially the definition of our softmax:

$$P(w|c)=\frac{\exp h^T v'_w}{\sum_{w_i∈V}\exp(h^⊤ v′_{w_i})}$$

For notational brevity and unambiguity, let us designate the denominator of the softmax with $Z(c)$, since the denominator only depends on $h$, which is generated from $c$ (assuming a fixed $V$). The softmax then looks like this:

$$P(w|c)=\frac{\exp h^T v'_w}{Z(c)}$$

Having to compute $P(w|c)$ means that -- again -- we need to compute $Z(c)$, which requires us to sum over the probabilities of all words in $V$. In the case of NCE, there exists a neat trick to circumvent this issue: We can treat the normalisation denominator $Z(c)$ as a parameter that the model can learn. 
Mnih and Teh (2012) and [Vaswani et al.](http://www.aclweb.org/anthology/D13-1140) actually keep $Z(c)$ fixed at 1, which they report does not affect the model's performance. This assumption has the nice side-effect of reducing the model's parameters, while ensuring that the model self-normalises by not depending on the explicit normalisation in $Z(c)$. Indeed, [Zoph et al.](http://www.aclweb.org/anthology/N16-1145.pdf) find that even when learned, $Z(c)$ is close to 1 and has low variance.

If we thus set $Z(c)$ to $1$ in the above softmax equation, we are left with the following probability of word $w$ given a context $c$:

$$P(w|c)=exp(h^⊤ v′_w)$$

Inserting this term in turn in our logistic regression objective finally yields the full NCE loss:

$$J_θ=−\sum_{w_i∈V}\left[\log\frac{\exp(h^⊤ v′_{w_i})}{\exp(h^⊤ v′_{w_i})+k Q(w_i)}+\sum^k_{j=1} \log(1−\frac{\exp(h^⊤ v′_{\tilde{w}_{ij}})}{exp(h^⊤ v′_{\tilde{w}_{ij}})+k Q(\tilde{w}_{ij}))}\right]$$

Note that NCE has nice theoretical guarantees: It can be shown that as we increase the number of noise samples k, the NCE derivative tends towards the gradient of the softmax function. [Mnih and Teh (2012)](https://www.cs.toronto.edu/~amnih/papers/ncelm.pdf) report that 25 noise samples are sufficient to match the performance of the regular softmax, with an expected speed-up factor of about 45. For more information on NCE, Chris Dyer has published some excellent [notes](https://arxiv.org/pdf/1410.8251.pdf).

One caveat of NCE is that as typically different noise samples are sampled for every training word w, the noise samples and their gradients cannot be stored in dense matrices, which reduces the benefit of using NCE with GPUs, as it cannot benefit from fast dense matrix multiplications. Jozefowicz et al. (2016) and Zoph et al. (2016) independently propose to share noise samples across all training words in a mini-batch, so that NCE gradients can be computed with dense matrix operations, which are more efficient on GPUs.

### Negative Sampling

Negative Sampling (NEG), the objective that has been popularised by [Mikolov et al. (2013)](https://arxiv.org/pdf/1301.3781), can be seen as an approximation to NCE. As we have mentioned above, NCE can be shown to approximate the loss of the softmax as the number of samples $k$ increases. NEG simplifies NCE and does away with this guarantee, as the objective of NEG is to learn high-quality word representations rather than achieving low perplexity on a test set, as is the goal in language modelling.

NEG also uses a logistic loss function to minimise the negative log-likelihood of words in the training set. Recall that NCE calculated the probability that a word $w$ comes from the empirical training distribution $P_{train}$ given a context $c$ as follows:

$$P(y=1|w,c)=\frac{\exp(h^⊤ v′_w)}{\exp(h^⊤ v′_w)+kQ(w)}$$

The key difference to NCE is that NEG only approximates this probability by making it as easy to compute as possible. For this reason, it sets the most expensive term, kQ(w) to 1, which leaves us with:

$$P(y=1|w,c)=\frac{\exp(h⊤v′w)}{\exp(h⊤v′w)+1}$$

kQ(w)=1 is exactly then true, when $k=|V|$ and $Q$ is a uniform distribution. In this case, NEG is equivalent to NCE. The reason we set $kQ(w)=1$ and not to some other constant can be seen by rewriting the equation, as $P(y=1|w,c)$ can be transformed into the sigmoid function:

$$P(y=1|w,c)=\frac{1}{1+exp(−h^⊤ v′_w)}$$

If we now insert this back into the logistic regression loss from before, we get:

$$J_θ = −\sum_{w_i∈V}\left[ \log{\frac{1}{1+\exp(−h^⊤ v′_{w_i})}} + \sum^k_{j=1} \log\left(1-\frac{1}{1+\exp{ (-h^T v'_{w_{ij}}) }}\right)\right]$$

By simplifying slightly, we obtain:

$$J_θ = −\sum_{w_i∈V}\left[ \log{\frac{1}{1+\exp(−h^⊤ v′_{w_i})}} + \sum^k_{j=1} \log \frac{1}{1+\exp{ (h^T v'_{w_{ij}}) }}\right]$$


Setting $σ(x)=\frac{1}{1+\exp(−x)} $finally yields the NEG loss:

$$J_θ=−\sum_{w_i∈V}\left[\log σ(h^⊤ v′_{w_i})+\sum^k_{j=1} \log σ(−h^⊤ v′_{\tilde{w}_{ij}})\right]$$

To conform with the notation of Mikolov et al. (2013), $h$ must be replaced with $v_{w_I}$, $v′_{w_i}$ with $v′_{w_O}$ and $v_{\tilde{w}_{ij}}$ with $v′_{w_i}$. Also, in contrast to Mikolov's NEG objective, we a) optimise the objective over the whole corpus, b) minimise negative log-likelihood instead of maximising positive log-likelihood (as mentioned before), and c) have already replaced the expectation $\mathbb{E}[\tilde{w}_{ik} \sim Q]$ with its Monte Carlo approximation. For more insights on the derivation of NEG, have a look at [Goldberg and Levy's notes](http://arxiv.org/abs/1402.3722).

We have seen that NEG is only equivalent to NCE when $k=|V|$ and $Q$ is uniform. In all other cases, NEG only approximates NCE, which means that it will not directly optimise the likelihood of correct words, which is key for language modelling. While NEG may thus be useful for learning word embeddings, its lack of asymptotic consistency guarantees makes it inappropriate for language modelling.

# Word2Vec with Gensim

More info: https://radimrehurek.com/gensim/models/word2vec.html

In [21]:
from gensim.models import Word2Vec
sentences = [["cat", "say", "meow"], ["dog", "say", "woof"]]
model = Word2Vec(sentences, min_count=1)

In [22]:
model.wv["cat"].shape

(100,)

In [50]:
from gensim.models import word2vec
from gensim.test.utils import datapath
sentences = word2vec.Text8Corpus('text8',max_sentence_length=10000)

In [52]:
for idx,sentence in enumerate(sentences):
    if idx==0:
        print(len(sentence))
print(idx)

10000
1700


In [67]:
model = Word2Vec(sentences, size=100, window=5, min_count=10, workers=4)

In [73]:
model.train(sentences, total_examples=model.corpus_count, epochs=10)

(123338263, 170052070)

In [75]:
model.wv.most_similar(["house"])

[('mansion', 0.6176140308380127),
 ('chamber', 0.6013445854187012),
 ('palace', 0.5718334913253784),
 ('manor', 0.5674700736999512),
 ('rooming', 0.5523747801780701),
 ('houses', 0.5498023629188538),
 ('commons', 0.5482867956161499),
 ('lords', 0.5352542400360107),
 ('usher', 0.5346463918685913),
 ('basement', 0.527206301689148)]

In [76]:
model.wv.most_similar(["boat"])

[('warship', 0.6990572214126587),
 ('ship', 0.6840620040893555),
 ('boats', 0.6776019334793091),
 ('helicopter', 0.6682183742523193),
 ('seaplane', 0.6619770526885986),
 ('truck', 0.6486808061599731),
 ('canoe', 0.6274817585945129),
 ('sailing', 0.627424955368042),
 ('hydrofoil', 0.6255449056625366),
 ('patrol', 0.621356725692749)]

In [78]:
analogy_scores = model.wv.accuracy(datapath('questions-words.txt'))

In [83]:
print(len(analogy_scores[0]["correct"]))
print(len(analogy_scores[0]["w"]))

298


## Word2Vec in Keras

More info in: http://adventuresinmachinelearning.com/word2vec-keras-tutorial/

In this section we are going to implement Word2Vec using text8.  
For this task, we must generate the positive and the negative samples.

### Positive Samples

The positive samples are the samples took from the empirical distribution $P_{train}$.

import helper
helper.get_random("articles_tom_",2010,field="body_norm",value={"$exists":1})

The field "body_norm" contains a list of normalized senteces generated from the "body" field. Since the body is pre-processed the following steps are:

1- Count al the $n$-grams in all the documents from 2010. We are going to use n=5.  
2- Keep only the $n$-grams that has a frequency greater than min_tok=100.  
3- Save the remaining n-grams in a collectio in mongoDB named "vocabulary_2010_2010".

The generated count of n-grams is stored in a document which "\_id" is "2010". The structure is detailed belown:

- \_id: year
- n-grams: number of n-grams to compute
- articles: number of articles processed
- size: total number of n-grams

In [1]:
from keras.models import Model
from keras.layers import Input, Dense, Reshape, dot
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence

import urllib
import collections
import os
import zipfile

import numpy as np
import tensorflow as tf

def maybe_download(filename, url, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urllib.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename


# Read the data into a list of strings.
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data


def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

def collect_data(vocabulary_size=10000):
    url = 'http://mattmahoney.net/dc/'
    filename = maybe_download('text8.zip', url, 31344016)
    vocabulary = read_data(filename)
    print(vocabulary[:7])
    data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                                vocabulary_size)
    del vocabulary  # Hint to reduce memory.
    return data, count, dictionary, reverse_dictionary

Using TensorFlow backend.


In [2]:
vocab_size = 10000
data, count, dictionary, reverse_dictionary = collect_data(vocabulary_size=vocab_size)
print(data[:7])

window_size = 3
vector_dim = 300
epochs = 2000000

valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

sampling_table = sequence.make_sampling_table(vocab_size)
couples, labels = skipgrams(data, vocab_size, window_size=window_size, sampling_table=sampling_table)
word_target, word_context = zip(*couples)
word_target = np.array(word_target, dtype="int32")
word_context = np.array(word_context, dtype="int32")

print(couples[:10], labels[:10])

Found and verified text8.zip
['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse']
[5234, 3081, 12, 6, 195, 2, 3134]
[[9372, 31], [3532, 216], [2172, 519], [1846, 2], [7037, 1], [8919, 26], [7419, 7303], [341, 26], [38, 3971], [1406, 8521]] [1, 1, 0, 1, 1, 1, 0, 1, 0, 0]


In [3]:
# create some input variables
input_target = Input((1,))
input_context = Input((1,))

embedding = Embedding(vocab_size, vector_dim, input_length=1, name='embedding')
target = embedding(input_target)
target = Reshape((vector_dim, 1))(target)
context = embedding(input_context)
context = Reshape((vector_dim, 1))(context)

# now perform the dot product operation to get a similarity measure
#dot_product = merge([target, context], mode='dot', dot_axes=1)
similarity = dot([target, context], normalize=True, axes=0)
dot_product = dot([target, context], normalize=False, axes=1)
dot_product = Reshape((1,))(dot_product)
# add the sigmoid output layer
output = Dense(1, activation='sigmoid')(dot_product)
# create the primary training model
model = Model(inputs=[input_target, input_context], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

# create a secondary validation model to run our similarity checks during training
validation_model = Model(inputs=[input_target, input_context], outputs=similarity)

In [ ]:
class SimilarityCallback:
    def run_sim(self):
        for i in range(valid_size):
            valid_word = reverse_dictionary[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            sim = self._get_sim(valid_examples[i])
            nearest = (-sim).argsort()[1:top_k + 1]
            log_str = 'Nearest to %s:' % valid_word
            for k in range(top_k):
                close_word = reverse_dictionary[nearest[k]]
                log_str = '%s %s,' % (log_str, close_word)
            print(log_str)

    @staticmethod
    def _get_sim(valid_word_idx):
        sim = np.zeros((vocab_size,))
        in_arr1 = np.zeros((1,))
        in_arr2 = np.zeros((1,))
        in_arr1[0,] = valid_word_idx
        for i in range(vocab_size):
            in_arr2[0,] = i
            out = validation_model.predict_on_batch([in_arr1, in_arr2])
            sim[i] = out
        return sim
sim_cb = SimilarityCallback()

arr_1 = np.zeros((1,))
arr_2 = np.zeros((1,))
arr_3 = np.zeros((1,))
for cnt in range(epochs):
    idx = np.random.randint(0, len(labels)-1)
    arr_1[0,] = word_target[idx]
    arr_2[0,] = word_context[idx]
    arr_3[0,] = labels[idx]
    loss = model.train_on_batch([arr_1, arr_2], arr_3)
    if cnt % 10000 == 0:
        print("Iteration {}, loss={}".format(cnt, loss))
    if cnt % 100000 == 0:
        sim_cb.run_sim()

Iteration 0, loss=0.6917519569396973
Nearest to these: sicily, cologne, read, eleven, enforced, floor, sometimes, benz,
Nearest to on: rival, unrest, attribute, contribute, saint, shifting, plots, top,
Nearest to nine: australian, qualified, transit, orbits, nodes, slight, suggestion, muscular,
Nearest to first: several, wheat, policy, dual, stands, relief, growing, release,
Nearest to often: ahead, jeff, especially, warrior, noting, deposits, somehow, collect,
Nearest to than: active, engineers, entertainment, moses, wearing, gaining, manifold, magical,
Nearest to it: expanded, commit, male, morning, manson, aphrodite, spontaneous, buying,
Nearest to have: confucius, magnus, gaming, lawyer, plan, confirmed, ka, flexible,
Nearest to its: abundance, theme, outside, discharge, feminist, instrumental, vehicles, personality,
Nearest to years: hour, boundary, describe, public, congressional, written, drew, equinox,
Nearest to history: spiritual, prevented, arrives, accounting, vita, ahmed, 

In [ ]:
sim_cb.run_sim()